# SETUP

In [1]:
import os
import re
import ast
import json
import ollama
import random
import base64
import requests
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from scipy import stats
from itertools import product
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# DF

In [ ]:
folder_path = r'\05_08_copilot_activities'

description_list = []

prompt_pattern = re.compile(r'(.*?)(?= --personalize| --style|$)', re.DOTALL)

for file_name in os.listdir(folder_path):
    if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff')):
        file_path = os.path.join(folder_path, file_name)
        
        image = Image.open(file_path)
        
        info = image.info
        
        description = info.get('Description', None)
        
        if description:
            match = prompt_pattern.match(description)
            cleaned_description = match.group(1) if match else description
        else:
            cleaned_description = None
        
        description_list.append({'filename': file_name, 'prompt': cleaned_description})

df = pd.DataFrame(description_list, columns=['filename', 'prompt'])

def clean_filename(filename):
    cleaned = re.sub(r'\d+', '', filename)
    cleaned = cleaned.replace('.jpeg', '').strip()
    return cleaned

df['prompt'] = df['filename'].apply(clean_filename)
df['codi'] = df['filename'].str.extract(r'(\d{2,3})')
df['age_group'] = df['prompt'].apply(lambda x: 'older' if 'older' in x else 'neutral')

df['ID_num'] = df['codi'].str.extract('(\d+)')
df['ID_num'] = pd.to_numeric(df['ID_num'])
df=df.sort_values(by='ID_num', ascending=True)

df = df.reset_index(drop=True)

df_older = df[df['age_group'] == 'older']
df_neutral = df[df['age_group'] == 'neutral']

df_older['ID'] = ''
for prompt in df_older['ID_num'].unique():
    df_subset = df_older[df_older['ID_num'] == prompt]
    count = 1
    for index, row in df_subset.iterrows():
        df_older.loc[index, 'ID'] = f'o_p{row["ID_num"]}_f{count}'
        count += 1

df_neutral['ID'] = ''
for prompt in df_neutral['ID_num'].unique():
    df_subset = df_neutral[df_neutral['ID_num'] == prompt]
    count = 1
    for index, row in df_subset.iterrows():
        df_neutral.loc[index, 'ID'] = f'a_p{row["ID_num"]}_f{count}'
        count += 1

In [16]:
df

,filename,prompt,codi,age_group,ID_num
0,01 walking to the bathroom older1.jpeg,walking to the bathroom older,01,older,1
1,01 walking to the bathroom older2.jpeg,walking to the bathroom older,01,older,1
2,01 walking to the bathroom person1.jpeg,walking to the bathroom person,01,neutral,1
3,03 moving between rooms person1.jpeg,moving between rooms person,03,neutral,3
4,03 walking to a room person1.jpeg,walking to a room person,03,neutral,3
...,...,...,...,...,...
769,137 home monitoring older4.jpeg,home monitoring older,137,older,137
770,137 home monitoring older3.jpeg,home monitoring older,137,older,137
771,137 home monitoring older2.jpeg,home monitoring older,137,older,137
772,137 home monitoring older1.jpeg,home monitoring older,137,older,137


In [17]:
output_dir = "C:/Users/David/Documents/AGEAI/Scripts/OUTPUTS/ANDREA/05_08_copilot_activities/CSV"

os.makedirs(output_dir, exist_ok=True)
df_older.to_csv(os.path.join(output_dir, "df_older.csv"), index=False)
df_neutral.to_csv(os.path.join(output_dir, "df_neutral.csv"), index=False)

In [ ]:
# Comprimir imagenes / AQUI NO CAL
def reduce_image_size(image, max_width, max_height):
    width, height = image.size
    if width > max_width or height > max_height:
        image.thumbnail((max_width, max_height), Image.LANCZOS)
    return image

def process_images(input_folder, neutral_df, older_df, neutral_subfolder, older_subfolder):
    neutral_output_folder = os.path.join(input_folder, neutral_subfolder)
    older_output_folder = os.path.join(input_folder, older_subfolder)
    
    if not os.path.exists(neutral_output_folder):
        os.makedirs(neutral_output_folder)
    if not os.path.exists(older_output_folder):
        os.makedirs(older_output_folder)
    
    neutral_files = neutral_df['filename'].tolist()
    older_files = older_df['filename'].tolist()
    
    for filename in os.listdir(input_folder):
        if filename.endswith('.png'):
            input_path = os.path.join(input_folder, filename)
            
            if filename in neutral_files:
                output_folder = neutral_output_folder
            elif filename in older_files:
                output_folder = older_output_folder
            else:
                print(f"Archivo no clasificado: {filename}")
                continue
            
            output_path = os.path.join(output_folder, os.path.splitext(filename)[0] + '.jpg')

            try:
                with Image.open(input_path) as img:
                    img = reduce_image_size(img, 1024, 1024)
                    img = img.convert('RGB')
                    img.save(output_path, 'JPEG', optimize=True, quality=85)
            except Exception as e:
                print(f"Error procesando {filename}: {e}")

    print("Proceso completado.")

input_folder = folder_path
neutral_subfolder = 'neutral_comp'
older_subfolder = 'older_comp'

process_images(input_folder, df_neutral, df_older, neutral_subfolder, older_subfolder)

In [ ]:
import os
import shutil

source_folder = "05_08_copilot_activities
destination_base_folder = source_folder

os.makedirs(os.path.join(destination_base_folder, 'older'), exist_ok=True)
os.makedirs(os.path.join(destination_base_folder, 'neutral'), exist_ok=True)

for index, row in df.iterrows():
    source_file = os.path.join(source_folder, row['filename'])
    
    destination_folder = os.path.join(destination_base_folder, row['age_group'])
    
    destination_file = os.path.join(destination_folder, row['filename'])
    
    if os.path.exists(source_file):
        shutil.copy(source_file, destination_file)
        print(f"Copiado: {source_file} a {destination_file}")
    else:
        print(f"No encontrado: {source_file}")

# OLLAMA

In [25]:
prompts = {
    "gender": "Answer only with the options provided: the gender of the person is: male, female, or if it's not possible to determine, none. Return answer as a JSON object {\"gender\": [\"gender\"]}.",
    "age_range": "Estimate the age range of the person in the image. Just answer with the following range of numbers: 18-24, 25-34, 35-44, 45-54, 55-64, 65-79 or 80+. Return answer only as a JSON object {\"age\": [\"range\"]}.",
    "race": "Estimate the race of the person in this image. Just answer with the following options: asian, black, white, hispanic, or other. Return answer only as a JSON object {\"race\": [\"race\"]}",
    "emotions_short": "Answer only with the options provided: the emotion expressed by the person is: neutral, positive, negative, or exaggerated. Return it as a JSON object {\"emotion\": [\"emotion\"]}",
    "personality_short": "Answer only with the options listed and no further information: the personality trait expressed by the person is: Openness, Conscientiousness, Extraversion, Agreeableness, or Neuroticism. Return it as a JSON object {\"personality\": [\"personality\"]}.",
    "location": "Answer only the options provided: The image is taken indoors, outdoors, or not possible to identify. Return it as a JSON object {\"location\": [\"location\"]} .",    
    "objects": "List up to 10 objects visible in this image. Return it as a JSON object as {\"objects\": [\"object\",...]}",
    "objects_assist_devices": "List any assistive devices visible in the image. Return it as a JSON object as {\"assistive_devices\": [\"object\",...]}",
    "objects_digi_devices": "List any digital devices visible in the image. Return it as a JSON object as {\"digital_devices\": [\"object\",...]}.",
    "person_count": "Answer only with the options provided: how many people can you count: 1, 2 or 3 or +3. Return answer as a JSON object {\"person_count\": [\"count\"]}.",
    "shot": "Answer only with the options provided: the type of shot in the image is: close-up shot, medium shot, or full shot. Return answer as a JSON object {\"shot\": [\"shot\"]}.",
    "position_short": "Describe the posture of the person in the image with one word. Be very concise and precise in you answer (standing, sitting, lying, etc.). Return answer as a JSON object {\"position\": [\"position\"]}.",
}

In [26]:
def call_ollama(image_path, prompt):
    try:
        with open(image_path, 'rb') as file:
            image_bytes = file.read()
        
        response = ollama.chat(
            model='llava',
            messages=[
                {
                    'role': 'user',
                    'content': prompt,
                    'images': [image_bytes],
                },
            ],
        )
        return response['message']['content']
    except Exception as e:
        print(f"Error calling Ollama: {e}")
        return "Error"

def process_image(image_path):
    try:
        results = {"ID_jpg": os.path.basename(image_path)}
        
        for prompt_name, prompt_text in prompts.items():
            results[prompt_name] = call_ollama(image_path, prompt_text)
        
        return results
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return {"ID_jpg": os.path.basename(image_path), "error": str(e)}

def process_folder(folder_path):
    all_results = []
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    for image_file in tqdm(image_files, desc="Processing images"):
        image_path = os.path.join(folder_path, image_file)
        results = process_image(image_path)
        all_results.append(results)
    
    return pd.DataFrame(all_results)

In [28]:
#NEUTRAL
df_neutral_results = process_folder("05_08_copilot_activities/neutral")
df_neutral_results.to_csv("05_08_copilot_activities/neutral/neutral_analysis_llava.csv", index=False)
print(f"Results saved")

Results saved


In [29]:
#OLDER
df_older_results = process_folder("05_08_copilot_activities/older")
df_older_results.to_csv("05_08_copilot_activities/older/older_analysis_llava.csv", index=False)
print(f"Results saved")

Processing images: 100%|██████████| 379/379 [59:46<00:00,  9.46s/it] 

Results saved


# MERGE FINAL

In [80]:
df_neutral = pd.read_csv(f'{output_dir}/df_neutral.csv')
df_older = pd.read_csv(f'{output_dir}/df_older.csv')
df_merged = pd.concat([df_neutral, df_older], axis=0)
df_neutral_cv = pd.read_csv(f'{output_dir}/neutral_analysis_llava.csv')
df_older_cv = pd.read_csv(f'{output_dir}/older_analysis_llava.csv')
df_merged_2 = pd.concat([df_neutral_cv, df_older_cv], axis=0)

df_merged_2 = df_merged_2.rename(columns={'ID_jpg': 'filename'})   

df_merged = pd.merge(df_merged, df_merged_2, on='filename', how='left')

In [83]:
def clean_json(text):
    if isinstance(text, str):
        cleaned_text = re.sub(r'```json', '', text)  
        cleaned_text = re.sub(r'```', '', cleaned_text)  
        
        cleaned_text = re.sub(r'\{\s+', '{', cleaned_text)
        cleaned_text = re.sub(r'\s+\}', '}', cleaned_text)
        
        cleaned_text = cleaned_text.replace('\n', '').strip()
        
        return cleaned_text
    return text

def ensure_list_format(value):
    if isinstance(value, str):
        try:
            data = ast.literal_eval(value)
            if isinstance(data, dict):
                for key in data:
                    if not isinstance(data[key], list):
                        data[key] = [data[key]]
                return str(data)
            return value
        except:
            return value
    return value

columns_to_transform = [
    'gender', 
    'age_range', 
    'race', 
    'emotions_short', 
    'location', 
    'personality_short', 
    'shot', 
    'position_short',
    'objects',
    'objects_assist_devices',
    'objects_digi_devices',
    'person_count'
]

for column in df_merged.columns:
    if column not in ['filename_png', 'filename_jpg']:
        df_merged[column] = df_merged[column].apply(clean_json)
        df_merged[column] = df_merged[column].apply(lambda x: x.lower() if isinstance(x, str) else x)
        df_merged[column] = df_merged[column].apply(ensure_list_format)

In [70]:
df_merged

,filename,prompt,codi,age_group,ID_num,ID,gender,age_range,race,emotions_short,personality_short,location,objects,objects_assist_devices,objects_digi_devices,person_count,shot,position_short
0,01 walking to the bathroom person1.jpeg,walking to the bathroom person,1,neutral,1,a_p1_f1,{'gender': ['female']},{'age': ['35-44']},{'race': ['asian']},{'emotion': ['positive']},{'personality': ['neuroticism']},{'location': ['indoor']},"{'objects': ['person', 'mug', 'newspaper', 'si...",{'assistive_devices': ['newspaper']},{'digital_devices': []},{'person_count': ['1']},{'shot': ['close-up shot']},{'position': ['standing']}
1,03 moving between rooms person1.jpeg,moving between rooms person,3,neutral,3,a_p3_f1,{'gender': ['female']},{'age': ['65-79']},{'race': ['white']},{'emotion': ['neutral']},{'personality': ['agreeableness']},{'location': ['indoor']},"{'objects': ['person', 'table', 'cup', 'tea po...",{'assistive_devices': ['cane']},{'digital_devices': ['none']},{'person_count': ['1']},{'shot': ['full shot']},{'position': ['standing']}
2,03 walking to a room person1.jpeg,walking to a room person,3,neutral,3,a_p3_f2,{'gender': ['female']},{'age': ['25-34']},{'race': ['white']},{'emotion': ['positive']},{'personality': ['openness']},{'location': ['outdoors']},"{'objects': ['woman', 'cat', 'backpack', 'door...",{'assistive_devices': []},{'digital_devices': ['cell phone']},{'person_count': ['1']},{'shot': ['medium shot']},{'position': ['standing']}
3,03 walking to a room person2.jpeg,walking to a room person,3,neutral,3,a_p3_f3,{'gender': ['male']},{'age': ['25-34']},{'race': ['asian']},{'emotion': ['positive']},{'personality': ['openness']},{'location': ['indoor']},"{'objects': ['person', 'coffee table', 'booksh...",{'assistive_devices': ['none']},{'digital_devices': []},{'person_count': ['1']},{'shot': ['medium shot']},{'position': ['standing']}
4,04 walking to a store person1.jpeg,walking to a store person,4,neutral,4,a_p4_f1,{'gender': ['male']},{'age': ['35-44']},{'race': ['black']},{'emotion': ['neutral']},{'personality': ['openness']},{'location': ['indoor']},{'objects': ['person with backpack and yellow ...,{'assistive_devices': []},{'digital_devices': ['phone']},{'person_count': ['2']},{'shot': ['medium shot']},{'position': ['standing']}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
769,136 at the hospital older1.jpeg,at the hospital older,136,older,136,o_p136_f4,{'gender': ['female']},{'age': ['65-79']},{'race': ['asian']},{'emotion': ['neutral']},{'personality': ['openness']},{'location': ['indoor']},"{'objects': ['person sitting in bed', ""book in...",{'assistive_devices': ['eyeglasses']},{'digital_devices': ['book']},{'person_count': ['1']},{'shot': ['close-up shot']},{'position': ['lying']}
770,137 home monitoring older4.jpeg,home monitoring older,137,older,137,o_p137_f1,{'gender': ['female']},{'age': ['65-79']},{'race': ['white']},{'emotion': ['neutral']},{'personality': ['agreeableness']},{'location': ['indoor']},"{'objects': ['older woman sitting in a chair',...",{'assistive_devices': ['tablet with a heart on...,"{'digital_devices': ['tablet', 'robot']}",{'person_count': ['1']},{'shot': ['close-up shot']},{'position': ['sitting']}
771,137 home monitoring older3.jpeg,home monitoring older,137,older,137,o_p137_f2,{'gender': ['female']},{'age': ['55-64']},{'race': ['white']},{'emotion': ['neutral']},{'personality': ['neuroticism']},{'location': ['indoor']},"{'objects': ['woman sitting on couch', 'coffee...",{'assistive_devices': ['robot']},"{'digital_devices': ['book', 'robot']}",{'person_count': ['2']},{'shot': ['full shot']},{'position': ['sitting']}
772,137 home monitoring older2.jpeg,home monitoring older,137,older,137,o_p137_f3,{'gender': ['female']},{'age': ['65-79']},{'race': ['white']},{'emotion': ['neutral']},{'personality': ['openness']},{'location': ['indoors']},"{'objects': ['person', 'tablet robot', 'coffee...",{'assistive_devices': ['robot']},"{'digital_devices': [{'type': 'robot'

In [ ]:
mapping = {
    #'{"location": ["indoor"]}': '{"location": ["indoors"]}',
    "{'location': ['indoor']}": "{'location': ['indoors']}",
    #'{"location": "indoors"}': '{"location": ["indoors"]}',
    #'{"location": "indoor"}': '{"location": ["indoors"]}',
    "{'location': ['indors']}": "{'location': ['indoors']}",
    "{'location': ['inddoors']}": "{'location': ['indoors']}",  
    "{'location': ['individual']}": "{'location': ['indoors']}",  
    "{'location': ['indicators']}": "{'location': ['indoors']}",  
}
df_merged['location'] = df_merged['location'].map(mapping).fillna(df_merged['location'])
df_merged['location'].unique()

mapping = {
    #'{"shot": "close-up shot"}': '{"shot": ["close-up shot"]}',
    #'{"shot": "full shot"}': '{"shot": ["full shot"]}',
    "{'shot': ['medium']}": "{'shot': ['medium shot']}",
    #'{"shot": ["medium"]}': '{"shot": ["medium shot"]}',
    #'{"shot": "medium shot"}': '{"shot": ["medium shot"]}',
    "{'shot': ['close-up']}": "{'shot': ['close-up shot']}",
}
df_merged['shot'] = df_merged['shot'].map(mapping).fillna(df_merged['shot'])
df_merged['shot'].unique()

#PERSON COUNT
def normalize_person_count(value):
    value = value.strip(".")
    
    value = re.sub(r'["]', "'", value)  
    value = re.sub(r"''", "'", value)   
    
    numbers = re.findall(r"'(\d+)'", value)
    if not numbers: 
        numbers = re.findall(r"(\d+)", value)
    
    if numbers:
        numbers = list(map(int, numbers))
        if any(num >= 3 for num in numbers):
            return "{'person_count': ['+3']}"
        else:
            return "{'person_count': ['" + "', '".join(map(str, numbers)) + "']}"
    else:
        return value

df_merged['person_count'] = df_merged['person_count'].apply(normalize_person_count)

mapping = {
    "{'person_count': [1]}": "{'person_count': ['1']}",
    "{'person_count': [2]}": "{'person_count': ['2']}",
    "{'person_count': [3]}": "{'person_count': ['3']}",
    "{'person_count': [''1'']}": "{'person_count': ['1']}",
    "{'person_count': [''2'']}": "{'person_count': ['2']}",
    "{'person_count': [1]}": "{'person_count': ['1']}",
    "{'person_count': [7]}": "{'person_count': ['+3']}",
    "{'person_count': ['1', '2']}": "{'person_count': ['+3']}",
}
df_merged['person_count'] = df_merged['person_count'].map(mapping).fillna(df_merged['person_count'])
df_merged['person_count'].unique()

In [87]:
def ensure_list(value):
    if isinstance(value, str):
        try:
            data = ast.literal_eval(value)
            if isinstance(data, dict):
                return list(data.values())[0] if isinstance(list(data.values())[0], list) else [list(data.values())[0]]
            elif isinstance(data, list):
                return data
        except:
            return [value]  
    return value if isinstance(value, list) else [value]  

def is_list(value):
    return isinstance(value, list)

columns_to_transform = [
    'gender',
    'age_range',
    'race', 
    'emotions_short', 
    'location', 
    'personality_short', 
    'shot', 
    'position_short',
    'objects',
    'objects_assist_devices',
    'objects_digi_devices',
    'person_count',
    'shot'
]

for column in columns_to_transform:
    df_merged[column] = df_merged[column].apply(ensure_list)

for column in columns_to_transform:
    if df_merged[column].apply(is_list).all():
        print(f"Todos los valores en la columna '{column}' son listas de Python.")
    else:
        print(f"Algunos valores en la columna '{column}' NO son listas de Python.")

def convert_assist_devices(value):
    if isinstance(value, list) or isinstance(value, str):
        if isinstance(value, list):
            value = str(value)

        if 'assistive_devices' in value:
            return ['none']
        
        if isinstance(value, str) and not value.strip():
            return ['none']

        try:
            data = ast.literal_eval(value)
            if isinstance(data, dict) and 'assistive_devices' in data:
                return ['none']
        except (ValueError, SyntaxError):
            pass

    if isinstance(value, list) and not value:
        return ['none']
    return value

df_merged['objects_assist_devices'] = df_merged['objects_assist_devices'].apply(convert_assist_devices)

def convert_digi_devices(value):
    if isinstance(value, list) or isinstance(value, str):
        if isinstance(value, list):
            value = str(value)

        if 'digital_devices' in value:
            return ['none']
        
        if isinstance(value, str) and not value.strip():
            return ['none']

        try:
            data = ast.literal_eval(value)
            if isinstance(data, dict) and 'digital_devices' in data:
                return ['none']
        except (ValueError, SyntaxError):
            pass

    if isinstance(value, list) and not value:
        return ['none']
    return value

df_merged['objects_digi_devices'] = df_merged['objects_digi_devices'].apply(convert_digi_devices)

def categorize_person_count(value):
    if isinstance(value, str):
        try:
            value = ast.literal_eval(value)
        except:
            return value
    
    if isinstance(value, list):
        if any(int(num) >= 3 for num in value):
            return ['+3']
        else:
            return [str(value[0])] 
    else:
        return value

df_merged['person_count'] = df_merged['person_count'].apply(categorize_person_count)

Todos los valores en la columna 'gender' son listas de Python.
Todos los valores en la columna 'age_range' son listas de Python.
Todos los valores en la columna 'race' son listas de Python.
Todos los valores en la columna 'emotions_short' son listas de Python.
Todos los valores en la columna 'location' son listas de Python.
Todos los valores en la columna 'personality_short' son listas de Python.
Todos los valores en la columna 'shot' son listas de Python.
Todos los valores en la columna 'position_short' son listas de Python.
Todos los valores en la columna 'objects' son listas de Python.
Todos los valores en la columna 'objects_assist_devices' son listas de Python.
Todos los valores en la columna 'objects_digi_devices' son listas de Python.
Todos los valores en la columna 'person_count' son listas de Python.
Todos los valores en la columna 'shot' son listas de Python.


# PIL BRIGHTNESS

In [89]:
def rgb_to_hex(rgb):
    return '#{:02x}{:02x}{:02x}'.format(int(rgb[0]), int(rgb[1]), int(rgb[2]))

def rgb_to_css(rgb):
    return f'rgb({int(rgb[0])}, {int(rgb[1])}, {int(rgb[2])})'

def simplify_color(rgb):
    color_names = {
        (0, 0, 0): "Negro", (255, 255, 255): "Blanco",
        (255, 0, 0): "Rojo", (0, 255, 0): "Verde", (0, 0, 255): "Azul",
        (255, 255, 0): "Amarillo", (255, 0, 255): "Magenta", (0, 255, 255): "Cian",
        (128, 128, 128): "Gris"
    }
    
    distances = {name: sum((a - b) ** 2 for a, b in zip(rgb, color_rgb))
                 for color_rgb, name in color_names.items()}
    return min(distances, key=distances.get)

def analizar_imagen(ruta_imagen, max_size=1000):
    try:
        with Image.open(ruta_imagen) as imagen:
            if max(imagen.size) > max_size:
                imagen.thumbnail((max_size, max_size))
            
            imagen_array = np.array(imagen)
        
        brillo_promedio = np.mean(imagen_array)
        
        contraste = np.std(imagen_array)
        
        if len(imagen_array.shape) == 3:  
            pixels = imagen_array.reshape(-1, 3)
            color_dominante, conteo = stats.mode(pixels, axis=0)
            color_dominante = color_dominante.ravel()
            conteo = conteo.ravel()[0]  
        else:  
            color_dominante = np.array([brillo_promedio] * 3)
            conteo = imagen_array.size
        
        total_pixels = imagen_array.size // 3 if len(imagen_array.shape) == 3 else imagen_array.size
        porcentaje_dominante = (conteo / total_pixels) * 100
        
        return {
            'filename': os.path.basename(ruta_imagen),
            'brillo_promedio': float(brillo_promedio),
            'contraste': float(contraste),
            'color_dominante': {
                'rgb': tuple(map(int, color_dominante)),
                'hex': rgb_to_hex(color_dominante),
                'css': rgb_to_css(color_dominante),
                'simplificado': simplify_color(tuple(map(int, color_dominante)))
            },
            'porcentaje_dominante': float(porcentaje_dominante)
        }
    except Exception as e:
        print(f"Error al procesar {os.path.basename(ruta_imagen)}: {str(e)}")
        return None

def analizar_carpeta(ruta_carpeta, limit=None):
    resultados = []
    archivos = [f for f in os.listdir(ruta_carpeta) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
    
    if limit:
        archivos = archivos[:limit]
    
    total = len(archivos)
    for i, archivo in enumerate(archivos, 1):
        ruta_completa = os.path.join(ruta_carpeta, archivo)
        resultado = analizar_imagen(ruta_completa)
        if resultado:
            resultados.append(resultado)
        print(f"Procesado {i}/{total}: {archivo}")
    
    return resultados

if __name__ == '__main__':
    resultados_n = analizar_carpeta("05_08_copilot_activities/neutral")
    resultados_o = analizar_carpeta("05_08_copilot_activities/older")
    
    for resultado_n in resultados_n:
        print(f"\nImagen: {resultado_n['filename']}")
        print(f"  Brillo promedio: {resultado_n['brillo_promedio']:.2f}")
        print(f"  Contraste: {resultado_n['contraste']:.2f}")
        print(f"  Color dominante:")
        print(f"    RGB: {resultado_n['color_dominante']['rgb']}")
        print(f"    Hex: {resultado_n['color_dominante']['hex']}")
        print(f"    CSS: {resultado_n['color_dominante']['css']}")
        print(f"    Simplificado: {resultado_n['color_dominante']['simplificado']}")
        print(f"  Porcentaje del color dominante: {resultado_n['porcentaje_dominante']:.2f}%")

    for resultado_o in resultados_o:
        print(f"\nImagen: {resultado_o['filename']}")
        print(f"  Brillo promedio: {resultado_o['brillo_promedio']:.2f}")
        print(f"  Contraste: {resultado_o['contraste']:.2f}")
        print(f"  Color dominante:")
        print(f"    RGB: {resultado_o['color_dominante']['rgb']}")
        print(f"    Hex: {resultado_o['color_dominante']['hex']}")
        print(f"    CSS: {resultado_o['color_dominante']['css']}")
        print(f"    Simplificado: {resultado_o['color_dominante']['simplificado']}")
        print(f"  Porcentaje del color dominante: {resultado_o['porcentaje_dominante']:.2f}%")

C:\Users\David\AppData\Local\Temp\ipykernel_16716\1780155287.py:33: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  color_dominante, conteo = stats.mode(pixels, axis=0)


Procesado 1/395: 01 walking to the bathroom person1.jpeg
Procesado 2/395: 03 moving between rooms person1.jpeg
Procesado 3/395: 03 walking to a room person1.jpeg
Procesado 4/395: 03 walking to a room person2.jpeg
Procesado 5/395: 04 strolling in the neighborhood person1.jpeg
Procesado 6/395: 04 strolling in the neighborhood person2.jpeg
Procesado 7/395: 04 walking to a store person1.jpeg
Procesado 8/395: 04 walking to a store person3.jpeg
Procesado 9/395: 05 climbing stairs person1.jpeg
Procesado 10/395: 05 climbing stairs person2.jpeg
Procesado 11/395: 06 walking in the metro person1.jpeg
Procesado 12/395: 06 walking in the metro person2.jpeg
Procesado 13/395: 07 cooking person1.jpeg
Procesado 14/395: 07 cooking person2.jpeg
Procesado 15/395: 07 cooking person3.jpeg
Procesado 16/395: 07 cooking person4.jpeg
Procesado 17/395: 08 dining at a restaurant person1.jpeg
Procesado 18/395: 08 dining at a restaurant person2.jpeg
Procesado 19/395: 08 dining at a restaurant person3.jpeg
Procesado

In [90]:
pil_n = pd.DataFrame(resultados_n)
pil_o = pd.DataFrame(resultados_o)
pil = pd.concat([pil_n, pil_o], axis=0)   
df_merged = df_merged.merge(pil, left_on='filename', right_on='filename', how='left')

In [91]:
df_merged.to_csv(f'{output_dir}/df_merged_copilot_19_8.csv', index=False)